In [1]:
import gadata_funcs,pdb,re
import pandas as pd
import numpy as np
import pylab as pl
from gadata_funcs import date2days,gadata_date2days,salesdata_date2days
from importlib import reload
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sys import stdout

%matplotlib notebook

In [2]:
# Loading in data and adding day info
df=pd.read_csv("gadata.csv",dtype={'date':str})
df2=pd.read_csv("/home/wlwoon/Dropbox/chicken_work/impersuasion_stuff/withSalesData/sales_order_line_timestamped.csv")
df['day']=df.date.apply(gadata_date2days)
df2['day']=df2.transaction_date.apply(salesdata_date2days)

In [3]:
# The ranges for the feature generate and target windows respectively
featLen=30
labelLen=20
featThreshold1=50
featThreshold2=100
featThreshold3=150

In [4]:
# Preparing to extract features
featThreshold=featThreshold3

inFeatRange=set(list(range(featThreshold-featLen,featThreshold)))
labelRange=set(list(range(featThreshold,featThreshold+labelLen)))

featdf=df[df.day.isin(inFeatRange)]
targetdf=df2[df2.day.isin(labelRange)]
targetdf=targetdf[targetdf.customer_id.isin(set(featdf.user_id))]

dg=featdf.groupby('user_id')
featdf.replace(np.nan,-10,inplace=True)

/home/wlwoon/anaconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# Feature extraction

# x (inputs)
xdf=dg.hits.count()
xdf=pd.concat((xdf,dg.country.first()),axis=1)
xdf['malaysia']=(xdf.country=='Malaysia').astype(int)
xdf['malsing']=(xdf.country.isin(['Malaysia','Singapore'])).astype(int)
xdf['meanDay']=dg.day.mean()
xdf['meanDuration']=dg.sessionDuration.mean()
xdf['maxDuration']=dg.sessionDuration.max()
xdf['meanPurchase']=dg.noPurchases.mean()
xdf['maxPurchase']=dg.noPurchases.max()
xdf['recency']=dg.day.apply(lambda tmp: np.min(featThreshold-tmp))
xdf['userType']=(dg.userType.first()=='Returning Visitor').astype(int)

# y (targets)
ydf=pd.DataFrame(xdf.index.isin(set(targetdf.customer_id)).astype(int),index=xdf.index)

In [9]:
# Just the relevant features, and formatting for easier use
x=xdf[['hits','meanPurchase']].values
y=ydf.values.reshape((-1,))

In [11]:
scores=[]
importances=[]
print("Cross validation round: ")
for count in range(10):
    stdout.write("#"+str(count)+",")
    
    #rf=GradientBoostingClassifier(n_estimators=300)
    #rf=XGBClassifier(n_estimators=250,max_depth=6)
    rf=RandomForestClassifier(n_estimators=350,max_depth=6)
    
    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.1)
    
    # Augmenting minority class
    minoritySamp=np.logical_and(ytrain==1,np.random.rand(len(ytrain))<.85)
    xtrain=np.concatenate((xtrain,xtrain[minoritySamp]),axis=0)
    ytrain=np.concatenate((ytrain,np.ones(np.sum(minoritySamp))),axis=0)
    
    rf.fit(xtrain,ytrain)
    importances.append(rf.feature_importances_)
    scores.append([accuracy_score(ytest,rf.predict(xtest)),precision_score(ytest,rf.predict(xtest)),recall_score(ytest,rf.predict(xtest))])

print()
scores=pd.DataFrame(scores,columns=['Accuracy','Precision','Recall'])
print(np.mean(scores))

Cross validation round: 
#0,#1,#2,#3,#4,#5,#6,#7,#8,#9,
Accuracy     0.721791
Precision    0.575146
Recall       0.660918
dtype: float64
